<a href="https://colab.research.google.com/github/MathBorgess/data_science_studies/blob/main/deep_learning/recurrent/sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

imdb = tf.keras.datasets.imdb

train_data, test_data = imdb.load_data()

Vou utilizar o dataset imdb para fazer a análise sentimental de reviews de filmes. O dataset é composto por 50 mil reviews de filmes, sendo 25 mil para treino e 25 mil para teste. Cada review é classificado como positivo ou negativo, e normalmente é utilizado com modelos de RNN e transformers, para análise sentimental.

In [2]:
import numpy as np
word_index = {k: (v+3) for k,v in imdb.get_word_index().items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for key, value in word_index.items()])

train_dataset_texts = []
train_dataset_labels = []
for index in range(len(train_data[0])):
    train_dataset_texts.append(' '.join([ reverse_word_index.get(i, '?') for i in train_data[0][index]]))
    train_dataset_labels.append(train_data[1][index])

test_dataset_texts = []
test_dataset_labels = []
for index in range(len(test_data[0])):
    test_dataset_texts.append(' '.join([ reverse_word_index.get(i, '?') for i in test_data[0][index]]))
    test_dataset_labels.append(test_data[1][index])


train_dataset = tf.data.Dataset.from_tensor_slices((train_dataset_texts, train_dataset_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_dataset_texts, test_dataset_labels))

pré-processamento do texto, mapeiando palavras para índexes e adicionando tokens especiais como <PAD>, <START>, e <UNK>. Depois, posso reconverter os índices das reviews de volta para texto e criar um dataset do TensorFlow (tf.data.Dataset), para melhor performance com GPU, com os textos e rótulos, preparando os dados para treinamento

In [3]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [4]:
vocab_size = 10000

encoder = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size)
encoder.adapt(train_dataset.map(lambda text, label: text))

2025-01-30 22:09:00.460413: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Como estou trabalhando com textos é importante que eu tenha um vocabulário de palavras, para isso vou utilizar o Tokenizer do Keras, que mapeia palavras para índices e vice-versa.

In [5]:
class CNNSentimentClassifier(tf.keras.Model):
    def __init__(self, encoder, num_filters=128, kernel_size=5, dense_units=64):
        super(CNNSentimentClassifier, self).__init__()
        self.encoder = encoder
        self.embedding = tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()),
                                                    output_dim=dense_units*2,
                                                    mask_zero=True)
        
        self.conv1 = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu')
        self.pool1 = tf.keras.layers.MaxPooling1D(pool_size=2)
        
        self.conv2 = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu')
        self.pool2 = tf.keras.layers.MaxPooling1D(pool_size=2)
        self.dense = tf.keras.layers.Dense(dense_units, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.embedding(x)
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        # x = tf.keras.layers.GlobalAveragePooling1D()(x)
        x = self.dense(x[:, -1, :])
        x = self.dropout(x)
        return self.output_layer(x)

Uma definição em classe bem simples, com layers de Embedding, Convolucional, pooling e Dense, para classificar os reviews como positivos ou negativos.
Muito importante mencionar o ajuste de dimensão [:,1,:], que é necessário para a entrada da camada Dense, que espera um tensor 2D.

In [6]:
model = CNNSentimentClassifier(encoder=encoder, dense_units=64)
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
model(np.array([sample_text]))
model.summary()

/Users/matheusborges/github/cin/data_science_studies/.venv/lib/python3.12/site-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "cnn_sentiment_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (1, None)              │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (1, None, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (1, None, 128)         │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (1, None, 128)         │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 64)                │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, 1)                 │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,452,417 (5.54 MB)

 Trainable params: 1,452,417 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.fit(train_dataset, epochs=5,
            validation_data=test_dataset,
            validation_steps=30)

Epoch 1/5


/Users/matheusborges/github/cin/data_science_studies/.venv/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:780: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


244/391 ━━━━━━━━━━━━━━━━━━━━ 23s 160ms/step - loss: 0.6935

Depois de compilar o modelo e treina-lo com o Binary Crossentropy, posso avaliar o modelo com o método evaluate, que retorna a acurácia do modelo.

In [ ]:
model.evaluate(test_dataset)

391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 78ms/step - loss: 0.6962


0.6953138113021851

Modelos de CNN são muito utilizados para análise de imagens, mas também podem ser utilizados para análise de textos, como é o caso desse exemplo, mesmo não sendo as queridinhas para esse tipo de tarefa de NLP